In [0]:
file_location = "dbfs:/user/hive/warehouse/insurance_claims"
file_type = "csv"

df = spark.read.format("delta").load(file_location, header=True, inferSchema=True)

In [0]:
df.head()

In [0]:
df.display

In [0]:
df.printSchema()

In [0]:
import pandas as pd
df.fraud_reported


In [0]:
df_pd = df.toPandas()
df_pd.head()

In [0]:
df.columns

In [0]:
df.show()

In [0]:
import pyspark.pandas as ps

In [0]:
df.select('fraud_reported').show()

In [0]:
df_pd['fraud_reported'].value_counts()

In [0]:
to_drop = ['policy_number','policy_bind_date','insured_zip','incident_location',
           'auto_year', 'auto_make', 'auto_model']
df_pd.drop(columns=to_drop, inplace=True)

In [0]:
#categorical to numeric

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in df_pd.columns:
    if df_pd[col].dtype == 'O':
        df_pd[col] = le.fit_transform(df_pd[col])

In [0]:
df_pd['fraud_reported'].values

In [0]:
from sklearn.ensemble import RandomForestClassifier
# split features and target
X = df_pd.iloc[:,:-1]              
Y = df_pd['fraud_reported']  

In [0]:
X

In [0]:
Y

In [0]:
from sklearn.model_selection import train_test_split
# ---------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
print(X_train.shape, X_test.shape)

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
lmodel = LogisticRegression()
lmodel.fit(X_train, y_train)

In [0]:
y_pred=lmodel.predict(X_test)

In [0]:
y_pred = pd.Series(y_pred)

In [0]:
y_test

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

report = (classification_report(y_test, y_pred))
report

In [0]:
cf_matrix = confusion_matrix(y_test, y_pred)
cf_matrix

In [0]:
import matplotlib.pyplot as plt
plt.plot(y_test,y_pred)